In [1]:
from functools import wraps, update_wrapper
from typing import Callable, Union

import numpy as np
import pandas as pd

%load_ext lab_black

In [2]:
def load_data(*, filename: str, sep: str = ",") -> pd.DataFrame:
    """This is used to load the data.

    Params;
        filename (str): The filepath.
        sep (str, default=","): The separator. e.g ',', '\t', etc

    Returns:
        data (pd.DataFrame): The loaded dataframe.
    """
    data = pd.read_csv(filename, sep=sep)
    print(f"Shape of data: {data.shape}\n")
    return data

In [3]:
# Load data
fp = "../../data/student-por.csv"
data = load_data(filename=fp, sep=";")

data.head()

Shape of data: (649, 33)



,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,0,11,13,13


### Creating Decorators

1.) Using function

```python
from functools import wraps


def outer(func): # decorator

    @wraps(func)
    def wrapper(*args, **kwargs):
        # Do something
        print("Decorator is decorating ...")
        return func(*args, **kwargs)

    return wrapper

```

2.) Using class

```python
class star:
    def __init__(self, func):
        self.func = func

    def inner(self, *args, **kwargs):
        # Do something
        print("Decorator is decorating ...")
        return self.func(*args, **kwargs)

    def __call__(self, *args, **kwargs):
        """This allows you to call the function"""
        return self.inner(*args, **kwargs)

```

In [4]:
_input = Union[np.ndarray, pd.DataFrame, pd.Series]


def shape_of_array(func: Callable) -> Callable:
    """This returns the shape of the NumPy array, Pandas
    DataFrame or Series."""

    @wraps(func)
    def wrapper(*args, **kwargs) -> str:
        input_ = func(*args, **kwargs)
        print(f"Shape of data: {input_.shape}\n")
        return input_

    return wrapper


def timer(func: Callable) -> Callable:
    """This is used to track the execution time of a function."""
    import time

    @wraps(func)
    def wrapper(*args, **kwargs) -> str:
        start_time = time.time()
        result = func(*args, **kwargs)
        stop_time = time.time()
        print(f"Duration: {round((stop_time - start_time), 3)} seconds")
        return result

    return wrapper

In [5]:
@timer
@shape_of_array
def preprocess_data(*, filename: str, sep: str = ",") -> pd.DataFrame:
    """This is used to preprocess the data.

    Params;
        filename (str): The filepath.
        sep (str, default=","): The separator. e.g ',', '\t', etc

    Returns:
        data (pd.DataFrame): The processed dataframe.
    """
    # Load the data, convert all column names to lower case and
    # one-hot encode the categorical columns (drop first=True)
    import time

    # Add latency
    time.sleep(2)

    data = pd.read_csv(filename, sep=sep)
    data.columns = data.columns.str.lower()
    data = pd.get_dummies(data, drop_first=True)
    return data

In [6]:
df = preprocess_data(filename=fp, sep=";")
df.head()

Shape of data: (649, 42)

Duration: 2.023 seconds


,age,medu,fedu,traveltime,studytime,failures,famrel,freetime,goout,dalc,...,guardian_mother,guardian_other,schoolsup_yes,famsup_yes,paid_yes,activities_yes,nursery_yes,higher_yes,internet_yes,romantic_yes
0,18,4,4,2,2,0,4,3,4,1,...,1,0,1,0,0,0,1,1,0,0
1,17,1,1,1,2,0,5,3,3,1,...,0,0,0,1,0,0,0,1,1,0
2,15,1,1,1,2,0,4,3,2,2,...,1,0,1,0,0,0,1,1,1,0
3,15,4,2,1,3,0,3,2,2,1,...,1,0,0,1,0,1,1,1,1,1
4,16,3,3,1,2,0,4,3,2,1,...,0,0,0,1,0,0,1,1,0,0


### Using Classes

In [7]:
class shape_of_array_class:
    """This returns the shape of the NumPy array, Pandas
    DataFrame or Series."""

    def __init__(self, func: Callable) -> None:
        self.func = func
        # Equivalent of @wraps(self.func)
        update_wrapper(self, func)

    def __call__(self, *args, **kwargs) -> str:
        input_ = self.func(*args, **kwargs)
        print(f"Shape of data: {input_.shape}\n")
        return input_


class timer_class:
    """This is used to track the execution time of a function."""

    def __init__(self, func: Callable) -> None:
        self.func = func
        # Equivalent of @wraps(self.func)
        update_wrapper(self, func)

    def __call__(self, *args, **kwargs) -> str:
        import time

        start_time = time.time()
        result = self.func(*args, **kwargs)
        stop_time = time.time()
        print(f"Duration: {round((stop_time - start_time), 3)} seconds")
        return result

In [8]:
@timer_class
@shape_of_array_class
def preprocess_data(*, filename: str, sep: str = ",") -> pd.DataFrame:
    """This is used to preprocess the data.

    Params;
        filename (str): The filepath.
        sep (str, default=","): The separator. e.g ',', '\t', etc

    Returns:
        data (pd.DataFrame): The processed dataframe.
    """
    # Load the data, convert all column names to lower case and
    # one-hot encode the categorical columns (drop first=True)
    import time

    # Add latency
    time.sleep(2)

    data = pd.read_csv(filename, sep=sep)
    data.columns = data.columns.str.lower()
    data = pd.get_dummies(data, drop_first=True)
    return data

In [9]:
df = preprocess_data(filename=fp, sep=";")
df.head()

Duration: 2.024 seconds


,age,medu,fedu,traveltime,studytime,failures,famrel,freetime,goout,dalc,...,guardian_mother,guardian_other,schoolsup_yes,famsup_yes,paid_yes,activities_yes,nursery_yes,higher_yes,internet_yes,romantic_yes
0,18,4,4,2,2,0,4,3,4,1,...,1,0,1,0,0,0,1,1,0,0
1,17,1,1,1,2,0,5,3,3,1,...,0,0,0,1,0,0,0,1,1,0
2,15,1,1,1,2,0,4,3,2,2,...,1,0,1,0,0,0,1,1,1,0
3,15,4,2,1,3,0,3,2,2,1,...,1,0,0,1,0,1,1,1,1,1
4,16,3,3,1,2,0,4,3,2,1,...,0,0,0,1,0,0,1,1,0,0
